In [ ]:
!pip install torchcam

Grad-CAM for Resnet18

In [ ]:
import torch
import torch.nn as nn
from torchvision import models


model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 15)


model.load_state_dict(torch.load("/content/resnet18_model-4.pth", map_location='cpu'))
model.eval()


In [ ]:
from torchcam.methods import GradCAM
from torchvision import transforms
from torchcam.utils import overlay_mask
from PIL import Image
import matplotlib.pyplot as plt


cam_extractor = GradCAM(model, target_layer='layer4')


img_path = "/content/598.jpg"  
img = Image.open(img_path).convert('RGB')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

input_tensor = transform(img).unsqueeze(0) 


out = model(input_tensor)
pred_class = out.argmax().item()


activation_map = cam_extractor(pred_class, out)


cam_tensor = activation_map[0].squeeze(0).numpy()
cam_image = Image.fromarray(cam_tensor, mode='F')


result = overlay_mask(img, cam_image, alpha=0.5)

plt.imshow(result)
plt.title(f"Grad-CAM for class {pred_class}")
plt.axis('off')
plt.show()



In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
from torchcam.methods import GradCAM
from torchcam.utils import overlay_mask
import matplotlib.pyplot as plt


model.eval()
cam_extractor = GradCAM(model, target_layer='layer4')  


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


input_folder = "/content/test_images"      
output_folder = "/content/gradcam_results"  

os.makedirs(output_folder, exist_ok=True)


for img_name in os.listdir(input_folder):
    if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        try:
          
            img_path = os.path.join(input_folder, img_name)
            img = Image.open(img_path).convert('RGB')
            input_tensor = transform(img).unsqueeze(0)

            
            out = model(input_tensor)

         
            pred_class = out.argmax().item()

        
            activation_map = cam_extractor(pred_class, out)


        
            cam_tensor = activation_map[0].squeeze(0).numpy()
            cam_image = Image.fromarray(cam_tensor, mode='F')

        
            result = overlay_mask(img, cam_image, alpha=0.5)

           
            save_path = os.path.join(output_folder, f"gradcam_{img_name}")
            result.save(save_path)
            print(f"[✓] Saved: {save_path}")
        except Exception as e:
            print(f"[✗] Failed on {img_name}: {e}")


Grad-CAM for Efficientnet_b0

In [ ]:
import torch
from torchvision import models
import torch.nn as nn


model = models.efficientnet_b0(pretrained=False)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 15)  


model.load_state_dict(torch.load("/content/efficientnet_b0_model.pth", map_location='cpu'))
model.eval()

In [ ]:
import os
from torchvision import transforms
from torchcam.methods import GradCAM
from torchcam.utils import overlay_mask
from PIL import Image
import matplotlib.pyplot as plt


cam_extractor = GradCAM(model, target_layer='features.7')  


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


input_folder = "/content/test_images"         
output_folder = "/content/efficientnet_cam"   
os.makedirs(output_folder, exist_ok=True)


for img_name in os.listdir(input_folder):
    if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        try:
            img_path = os.path.join(input_folder, img_name)
            img = Image.open(img_path).convert('RGB')
            input_tensor = transform(img).unsqueeze(0)

          
            out = model(input_tensor)
            pred_class = out.argmax().item()

       
            activation_map = cam_extractor(pred_class, out)
            cam_tensor = activation_map[0].squeeze(0).numpy()
            cam_image = Image.fromarray(cam_tensor, mode='F')

       
            result = overlay_mask(img, cam_image, alpha=0.5)

          
            save_path = os.path.join(output_folder, f"cam_{img_name}")
            result.save(save_path)
            print(f"[✓] Saved: {save_path}")
        except Exception as e:
            print(f"[✗] Failed on {img_name}: {e}")
